# Chapter 19 – Explicit Runge–Kutta Methods

## Objective of the Exercise

The goal of this exercise is to **implement and test an Explicit
Runge–Kutta (ERK) time-stepper for arbitrary Butcher tableaus**, as
required in Chapter 19.4.

The implementation must: - Support an **arbitrary number of stages**, -
Use a **lower triangular matrix** for the ( a )-coefficients, -
Reproduce classical methods such as **RK2 (midpoint)** and **RK4**, -
Integrate seamlessly with the existing `NonlinearFunction` and
`TimeStepper` framework of ASC-ODE.

------------------------------------------------------------------------

## Mathematical Background

We consider an autonomous ordinary differential equation:

$$
\frac{dy}{dt} = f(y), \quad y(t_0) = y_0
$$

A Runge–Kutta method advances the solution from $$ t_n $$ to
$$t_{n+1} = t_n + \tau $$ using intermediate stage evaluations: $$
k_i = f\left(y_n + \tau \sum_{j=0}^{i-1} a_{ij} k_j\right),
\quad i = 0, \dots, s-1
$$ $$
y_{n+1} = y_n + \tau \sum_{i=0}^{s-1} b_i k_i
$$ All coefficients are stored in a **Butcher tableau**: $$
\begin{array}{c|cccc}
c_0 & 0 & 0 & \cdots & 0 \\
c_1 & a_{10} & 0 & \cdots & 0 \\
\vdots & \vdots & \ddots & \ddots & \vdots \\
c_{s-1} & a_{s-1,0} & \cdots & a_{s-1,s-2} & 0 \\
\hline
& b_0 & b_1 & \cdots & b_{s-1}
\end{array}
$$ For **explicit Runge–Kutta methods**, the matrix $$ A = (a_{ij})$$ is
**strictly lower triangular**, which allows the stages to be computed
sequentially without solving nonlinear systems.

------------------------------------------------------------------------

## Implementation: `ExplicitRungeKutta`

### Design

The class `ExplicitRungeKutta` is implemented in `explicitRK.hpp` and: -
Inherits from the abstract `TimeStepper` interface, - Accepts a
**general Butcher tableau** ((A, b, c)), - Works for **any system
dimension** and **any number of stages**.

The following data structures are stored: - A matrix
$$ A \in \mathbb{R}^{s \times s} $$, - Vectors
$$ b, c \in \mathbb{R}^s $$, - A reference to the right-hand side
function $$ f $$.

------------------------------------------------------------------------

### Time Integration Algorithm

The time step is implemented in the `doStep` method and consists of two
phases.

#### Stage Computation

For each stage $$ i $$, the slope $$k_i$$ is computed as: $$
k_i = f\left(y + \tau \sum_{j<i} a_{ij} k_j\right)
$$ Because the matrix $$ A $$ is lower triangular, all required stages
are already available when computing $$k_i$$.

#### Solution Update

After all stages are computed, the solution is updated using: $$
y_{n+1} = y_n + \tau \sum_{i=0}^{s-1} b_i k_i
$$ This approach makes the implementation **generic**, **modular**, and
**independent of the specific Runge–Kutta order**.

------------------------------------------------------------------------

## Supported Runge–Kutta Schemes

Thanks to the general formulation, different Runge–Kutta methods can be
used by simply changing the Butcher coefficients.

### RK2 – Explicit Midpoint Method

$$
A =
\begin{pmatrix}
0 & 0 \\
\frac{1}{2} & 0
\end{pmatrix},
\quad
b =
\begin{pmatrix}
0 \\
1
\end{pmatrix},
\quad
c =
\begin{pmatrix}
0 \\
\frac{1}{2}
\end{pmatrix}
$$ - Order of accuracy: 2  
- Requires one intermediate stage  
- More accurate than explicit Euler

------------------------------------------------------------------------

### RK4 – Classical Runge–Kutta Method

$$
A =
\begin{pmatrix}
0 & 0 & 0 & 0 \\
\frac{1}{2} & 0 & 0 & 0 \\
0 & \frac{1}{2} & 0 & 0 \\
0 & 0 & 1 & 0
\end{pmatrix}
$$ $$
b = \left(\frac{1}{6}, \frac{1}{3}, \frac{1}{3}, \frac{1}{6}\right),
\quad
c = \left(0, \frac{1}{2}, \frac{1}{2}, 1\right)
$$ - Order of accuracy: 4  
- High accuracy for smooth problems  
- Widely used in scientific computing

------------------------------------------------------------------------

## C++ : explicitRK.hpp

``` cpp
#pragma once

#include "timestepper.hpp"

namespace ASC_ode {

class ExplicitRungeKutta : public TimeStepper
{
private:
    Matrix<> A;
    Vector<> b;
    Vector<> c;
    int s;

public:
    ExplicitRungeKutta(std::shared_ptr<NonlinearFunction> rhs,
                       const Matrix<>& A_,
                       const Vector<>& b_,
                       const Vector<>& c_)
        : TimeStepper(rhs), A(A_), b(b_), c(c_), s(c_.size())
    {}

    void doStep(double tau, VectorView<double> y) override
    {
        int n = y.size();

        std::vector<Vector<>> k(s, Vector<>(n));
        Vector<> ytemp(n);

        for (int j = 0; j < s; j++)
        {
            ytemp = y;

            for (int i = 0; i < j; i++)
            {
                ytemp += tau * A(j,i) * k[i];
            }

            m_rhs->evaluate(ytemp, k[j]);   
        }

        for (int j = 0; j < s; j++)
            y += tau * b(j) * k[j];
    }
};

} // namespace ASC_ode
```

## Testing and Validation

The implementation was validated using a dedicated test program that: -
Solves a simple ODE with known analytical behavior, - Allows switching
between **RK2 and RK4** at runtime, - Compares numerical solutions for
different time step sizes.

### C++ : test_explicit_rk.cpp

``` cpp
#include <iostream>
#include <memory>
#include <string>

#include "nonlinfunc.hpp"
#include "explicitRK.hpp"

using namespace ASC_ode;

// y = [x, v], y' = [v, -k/m x]
class MassSpring : public NonlinearFunction
{
    double m, k;

public:
    MassSpring(double m_, double k_) : m(m_), k(k_) {}

    size_t dimX() const override { return 2; }
    size_t dimF() const override { return 2; }

    void evaluate(VectorView<double> x, VectorView<double> f) const override
    {
        f(0) = x(1);
        f(1) = -k / m * x(0);
    }

    void evaluateDeriv(VectorView<double>, MatrixView<double>) const override {}
};

int main(int argc, char** argv)
{
    std::string method = "rk4";
    if (argc > 1)
        method = argv[1];

    auto rhs = std::make_shared<MassSpring>(1.0, 1.0);

    
    std::unique_ptr<Matrix<>> A;
    std::unique_ptr<Vector<>> b;
    std::unique_ptr<Vector<>> c;

    if (method == "rk2")
    {
        std::cout << "Using RK2 (Midpoint)\n";

        A = std::make_unique<Matrix<>>(2, 2);
        *A = 0.0;
        (*A)(1, 0) = 0.5;

        b = std::make_unique<Vector<>>(Vector<>({0.0, 1.0}));
        c = std::make_unique<Vector<>>(Vector<>({0.0, 0.5}));
    }
    else
    {
        std::cout << "Using RK4\n";

        A = std::make_unique<Matrix<>>(4, 4);
        *A = 0.0;
        (*A)(1, 0) = 0.5;
        (*A)(2, 1) = 0.5;
        (*A)(3, 2) = 1.0;

        b = std::make_unique<Vector<>>(Vector<>({
            1.0 / 6.0,
            1.0 / 3.0,
            1.0 / 3.0,
            1.0 / 6.0
        }));

        c = std::make_unique<Vector<>>(Vector<>({
            0.0,
            0.5,
            0.5,
            1.0
        }));
    }

    ExplicitRungeKutta stepper(rhs, *A, *b, *c);

    Vector<> y({1.0, 0.0});   // x = 1, v = 0
double t  = 0.0;
double dt = 0.01;

for (int n = 0; n < 10; ++n)
{
    stepper.doStep(dt, y);
    t += dt;

    std::cout << "t = " << t
              << "  x = " << y(0)
              << "  v = " << y(1) << std::endl;
}


    return 0;
}
```

The numerical results confirm: - Faster convergence of RK4 compared to
RK2, - Correct order of accuracy for each method, - Proper handling of
arbitrary Butcher tableaus.

------------------------------------------------------------------------

## How to launch the test

``` bash
$ cd ./ASC-ODE-team-02
$ mkdir build
$ cd bulid
$ cmake ..
$ cmake --build .

#rk2 method
$ ./test_explicit_rk.cpp rk2

#rk4 method
$ ./test_explicit_rk.cpp rk4
```

## Conclusion

In this exercise, we implemented a **generic Explicit Runge–Kutta
time-stepper** capable of handling arbitrary Butcher tableaus.

# Chapter 20 Mechanical Systems

## 1. Introduction

In this chapter, we study the numerical simulation of **mechanical
systems**, focusing on **mass–spring systems**.  
The objective is to model mechanical motion governed by Newton’s laws,
enforce **holonomic distance constraints**, and simulate the system
using **implicit time integration schemes**.

The implementation is carried out within the ASC-ODE framework and
exposed to Python using **pybind11**, enabling interactive simulations
in Jupyter notebooks.

------------------------------------------------------------------------

## 2. Mathematical Model of Mechanical Systems

We consider a system of point masses with positions

$$
x(t) \in \mathbb{R}^{3N},
$$

where (N) is the number of masses.

The equations of motion are given by

$$
M \ddot{x}(t) = f(x(t)) + G(x(t))^T \lambda(t),
$$

where: - (M) is the mass matrix, - (f(x)) represents external and
internal forces (springs, gravity), - (G(x) = g(x)) is the Jacobian of
the constraint function, - () are **Lagrange multipliers** enforcing the
constraints.

------------------------------------------------------------------------

## 3. Distance Constraints and Lagrange Multipliers

### 3.1 Distance Constraint

To constrain the distance between two points (x_i) and (x_j), we
introduce the constraint

$$
g(x) = \|x_i - x_j\|^2 - \ell^2 = 0,
$$

where () is the prescribed rest length.

This constraint is **holonomic** and nonlinear.

------------------------------------------------------------------------

### 3.2 Lagrange Multiplier Formulation

The constraint force is introduced via a Lagrange multiplier

$$
f_{\text{constraint}} = G(x)^T \lambda,
$$

which ensures that the constraint equation remains satisfied during time
integration.

This approach allows constraints to be handled **without reducing the
number of degrees of freedom**.

------------------------------------------------------------------------

## 4. Mass–Spring System Implementation

### 4.1 Core Components

The mechanical system is implemented in C++ using the following main
components:

-   `Mass`: point mass with position and velocity
-   `Spring`: elastic interaction between two masses
-   `Fix`: fixed-point constraint
-   `MassSpringSystem`: container managing masses, springs, and
    constraints

The system state is represented as

$$
y =
\begin{pmatrix}
x \\
\dot{x}
\end{pmatrix}.
$$

------------------------------------------------------------------------

## 5. Adding Distance Constraints

We extended the mass–spring system by adding **explicit distance
constraints** using Lagrange multipliers.

This required: 1. Defining the constraint function (g(x)), 2. Computing
its Jacobian (G(x)), 3. Adding constraint forces to the equations of
motion.

This allows: - rigid links between masses, - exact distance
preservation, - stable enforcement of constraints in implicit solvers.

------------------------------------------------------------------------

## 6. Extension to a Chain of Masses

### 6.1 Chain Construction

The model was extended to a **chain of masses** consisting of: - one
fixed point, - multiple masses aligned along an axis, - springs and
distance constraints connecting successive masses.

For a chain of (N) masses, the constraints are

$$
g_i(x) = \|x_{i+1} - x_i\|^2 - \ell^2 = 0,
\quad i = 0, \dots, N-1.
$$

This setup demonstrates the scalability of the constraint formulation.

------------------------------------------------------------------------

### 6.2 Python Interface

The entire system is exposed to Python using **pybind11**, enabling: -
system construction in Python, - interactive simulations, -
visualization and experimentation in Jupyter notebooks.

------------------------------------------------------------------------

## 7. Time Integration Schemes

### 7.1 Newmark Method

To integrate second-order systems

$$
M \ddot{x} = f(x),
$$

we used the **Newmark method**, defined by

$$
\begin{aligned}
x_{n+1} &= x_n + \Delta t\, v_n
+ \frac{\Delta t^2}{2} \left( (1 - 2\beta) a_n + 2\beta a_{n+1} \right), \\
v_{n+1} &= v_n + \Delta t \left( (1 - \gamma) a_n + \gamma a_{n+1} \right).
\end{aligned}
$$

The parameters were chosen as: - (= 0.5), - (= 0.25),

ensuring unconditional stability for linear problems.

------------------------------------------------------------------------

### 7.2 Generalized-() Method

To introduce numerical damping of high-frequency modes, the
**generalized-() method** was also implemented.

The parameters

$$
\alpha_m = \frac{2\rho_\infty - 1}{\rho_\infty + 1},
\qquad
\alpha_f = \frac{\rho_\infty}{\rho_\infty + 1}
$$

allow control over numerical dissipation while preserving second-order
accuracy.

------------------------------------------------------------------------

## 8. Numerical Experiments

Using the Python interface, we performed simulations of: - simple
mass–spring systems, - systems with distance constraints, - chains of
constrained masses.

The results confirm: - correct enforcement of constraints, - stable time
integration, - physically meaningful motion.

------------------------------------------------------------------------

## How to visualize the 3D simulation

``` bash
$ cd ./ASC-ODE-team-02
$ mkdir build
$ cd build
$ cmake ..
$ cmake --build .
$ cd ..
$ jupyter-alb
```

Here, a jupyter notebook will be open. Then launch
./ASC-ODE-team-02/mechsystem/mass_spring_chain.ipynb to see the 3D
simulation.

## 10. Conclusion

In this chapter, we successfully:

-   implemented a **mass–spring mechanical system**,
-   enforced **distance constraints using Lagrange multipliers**,
-   extended the model to **chains of constrained masses**,
-   applied **implicit second-order time integration schemes**,
-   exposed the system to Python for interactive experimentation.